In [5]:
import weave
import os

os.environ["WF_TRACE_SERVER_URL"] = "http://127.0.01:6345"

client = weave.init("action_test")

In [ ]:
# from weave.trace import autopatch, weave_init
# from weave.trace_server import clickhouse_trace_server_batched

# ch_server = clickhouse_trace_server_batched.ClickHouseTraceServer.from_env()
# inited_client = weave_init.InitializedClient(client)
# autopatch.autopatch()


In [6]:
from openai import OpenAI

openai_client = OpenAI()

@weave.op
def extract_name(user_input: str) -> str:
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Extract the name from the user input. If there is no name, return an empty string.",
            },
            {"role": "user", "content": user_input},
        ],
        temperature=0.0,
        max_tokens=64,
        top_p=1,
    )
    return response.choices[0].message.content

calls = []
for i in range(10):
    res, call = extract_name.call(f"My name is Tim {i}.")
    calls.append(call)

print(calls)

🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-ac22-7371-8673-e42ab45a17c8
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-ae1f-7572-a5f8-e0c0020f4094
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-af92-7731-aec1-d38da032bd27
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-b0fc-7bd0-b16f-3ece32a43a4d
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-b26b-7392-9522-a26135d4ea8a
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-b3be-7320-b52f-2213225e0a84
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-b513-74c2-9346-d358060e321b
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-b60a-7d43-9453-d45cd6955ba4
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-b6f9-7303-a7b4-593ab39120b5
🍩 https://wandb.ai/timssweeney/action_test/r/call/0192b281-b868-7a43-8f78-312fd7201a45
[Call(_op_name=<Future at 0x2be384a60 state=finished returned str>, trace_id='0192b281-ac22-7371-8673-e439cb4bc713', project_id='t

In [3]:
from weave.flow import action_objects
from weave.trace.weave_client import get_ref
from weave.trace_server.interface import actions

action = action_objects.ActionWithConfig(
    name="is_name_extracted",
    action=actions.BuiltinAction(
        name="openai_completion",
    ),
    config={
        "model": "gpt-4o-mini",
        "system_prompt": "Given the following prompt and response, determine if the name was extracted correctly.",
        "response_format": {
            "type": "json_schema",
            "json_schema": {
                "name": "is_name_extracted",
                "schema": {
                    "type": "object",
                    "properties": {"is_extracted": {"type": "boolean"}},
                    "required": ["is_extracted"],
                    "additionalProperties": False,
                },
                "strict": True,
            },
        },
    },
)
mapping = action_objects.ActionOpMapping(
    action=action,
    op_name=get_ref(extract_name).name,
    op_digest=get_ref(extract_name).digest,
    input_mapping={
        "prompt": "inputs.user_input",
        "response": "output",
    },
)

In [4]:

from weave.trace_server import trace_server_interface as tsi

res = client.server.execute_batch_action(req=tsi.ExecuteBatchActionReq(
    project_id=client._project_id(), call_ids=[c.id for c in calls], mapping=mapping
))

HTTPError: 404 Client Error: Not Found for url: https://trace.wandb.ai/batch_action/execute